In [21]:
from sklearn import metrics
from scipy.sparse import hstack
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from sklearn.metrics import roc_auc_score
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm 
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

In [22]:
data=pd.read_csv("title_text_user.csv")

In [23]:
data

,Unnamed: 0,index,user,gender,ideology,religion,ethnicity,title,side,title_final
0,0,0,0905845,Female,Other,Christian,White,Rap Battle 5 Cheesedingo VS 0905845,Con,"['rap', 'battle', 'cheesedingo', 'v']"
1,1,1,091915,Female,Other,Christian,White,does everything revolves around math?,Con,"['everything', 'revolves', 'around', 'math']"
2,2,2,10438,Female,Other,Christian,White,Pageants are STUPID.,Con,"['pageant', 'stupid']"
3,3,3,105060,Male,Other,Non Believer,White,Is it possible to create an actual utopia?,Pro,"['possible', 'create', 'actual', 'utopia']"
4,4,4,11BangBangEnvyme,Male,Right,Christian,White,Obama is the best choice for president.,Pro,"['obama', 'best', 'choice', 'president']"
...,...,...,...,...,...,...,...,...,...,...
3961,3961,3961,zneuser93,Male,Center,Christian,White,My hypothetical group of dead musicians is bet...,Con---Pro---Con---Pro,"['hypothetical', 'group', 'dead', 'musician', ..."
3962,3962,3962,zobothehobo,Female,Left,Non Believer,Middle Eastern,Texting in school: is it THAT bad?,Con,"['texting', 'school', 'bad']"
3963,3963,3963,zoetwinn,Female,Left,Non Believer,White,The Royal Family are an asset to the UK,Con,"['royal', 'family', 'asset', 'uk']"
3964,3964,3964,zorasbrown,Male,Center,Christian,Asian,if theres no clock( or watch/sundials/wrist wa...,Con---Pro,"['theres', 'clock', 'time', 'recorder', 'time'..."


In [24]:
#splitting into train/eval/test set
dataset_train = data[:int(0.8*len(data))]
dataset_eval = data[-int(0.2*len(data)):-int(0.1*len(data))]
dataset_test = data[-int(0.1*len(data)):]

In [25]:
#Encoding
Encoder = LabelEncoder()

#word vectorization
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data["title_final"])
Train_X_Tfidf_title = Tfidf_vect.transform(dataset_train["title_final"])
Test_X_Tfidf_title = Tfidf_vect.transform(dataset_test["title_final"])

#encoding the side
Train_X_1_pos = Encoder.fit_transform(dataset_train['side'])
Test_X_1_pos = Encoder.fit_transform(dataset_test['side'])

#stacking everything together
X_train_tfidf = hstack([Train_X_Tfidf_title, Train_X_1_pos.reshape(-1, 1)])
X_test_tfidf = hstack([Test_X_Tfidf_title, Test_X_1_pos.reshape(-1, 1)])

#Label encode the target variable — 
#This is done to transform Categorical data of string type in the data set 
#into numerical values which the model can understand.

#gender
Train_Y_gender = Encoder.fit_transform(dataset_train['gender'])
Test_Y_gender = Encoder.fit_transform(dataset_test['gender'])

#religion
Train_Y_rel = Encoder.fit_transform(dataset_train['religion'])
Test_Y_rel = Encoder.fit_transform(dataset_test['religion'])

#ethnicity
Train_Y_ethn = Encoder.fit_transform(dataset_train['ethnicity'])
Test_Y_ethn = Encoder.fit_transform(dataset_test['ethnicity'])

#ideology
Train_Y_id = Encoder.fit_transform(dataset_train['ideology'])
Test_Y_id = Encoder.fit_transform(dataset_test['ideology'])

#stacking into a vector
y_train=np.stack((Train_Y_gender,Train_Y_rel,Train_Y_ethn,Train_Y_id),axis=1)
y_test=np.stack((Test_Y_gender,Test_Y_rel,Test_Y_ethn,Test_Y_id),axis=1)

In [ ]:
#using we can use the argument class_weight='balanced'  
#to penalize mistakes on the minority class by an amount proportional to how under-represented it is.
SVM = svm.SVC(C=1.0, kernel='linear',class_weight='balanced',gamma='auto',probability=True)
# Create the Multioutput Classifier
mor = MultiOutputClassifier(SVM)
# Train the regressor
mor.fit(X_train_tfidf, y_train)
# Generate predictions for testing data
y_pred = mor.predict(X_test_tfidf)



In [ ]:
print("SVM Metrics for Gender")
print(f"The F1 score is {f1_score(y_test[:,0], y_pred[:,0], average='weighted')}")

In [ ]:
print("SVM Metrics for Religion")
print(f"The F1 score is {f1_score(y_test[:,1], y_pred[:,1], average='weighted')}")

In [ ]:
print("SVM Metrics for Ethnicity")
print(f"The F1 score is {f1_score(y_test[:,2], y_pred[:,2], average='weighted')}")

In [ ]:
print("SVM Accuracy Score Ideology")
print(f"The F1 score is {f1_score(y_test[:,3], y_pred[:,3], average='weighted')}")

In [ ]:
 ## Comparing long and short titles

In [ ]:
def length(x):
    l=0
    for i in x:
        for j in i:
            l+=len(j)
            
    return l

data["text_length"]=data["title"].apply(length)

In [ ]:
val_med=int(np.median(data["text_length"]))

#dividing the dataset into two model
#Model 1: text_length>val_med
#Model 1: text_length>val_med

per_users_high=data[data["text_length"]>val_med].reset_index()
per_users_low=data[data["text_length"]<val_med].reset_index()
#splitting into train/eval/test set
dataset_train_long = per_users_high[:int(0.8*len(per_users_high))]
dataset_eval_long = per_users_high[-int(0.2*len(per_users_high)):-int(0.1*len(per_users_high))]
dataset_test_long = per_users_high[-int(0.1*len(per_users_high)):]
#splitting into train/eval/test set
dataset_train_short = per_users_low[:int(0.8*len(per_users_low))]
dataset_eval_short = per_users_low[-int(0.2*len(per_users_low)):-int(0.1*len(per_users_low))]
dataset_test_short = per_users_low[-int(0.1*len(per_users_low)):]

In [ ]:
Encoder = LabelEncoder()

#word vectorization
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(per_users_high["title_final"])
Train_X_Tfidf_title_long = Tfidf_vect.transform(dataset_train_long["title_final"])
Test_X_Tfidf_title_long = Tfidf_vect.transform(dataset_test_long["title_final"])

#encoding the side
Train_X_1_pos = Encoder.fit_transform(dataset_train_long['side'])
Test_X_1_pos = Encoder.fit_transform(dataset_test_long['side'])

#stacking everything together
X_train_tfidf = hstack([Train_X_Tfidf_title_long, Train_X_1_pos.reshape(-1, 1)])
X_test_tfidf = hstack([Test_X_Tfidf_title_long, Test_X_1_pos.reshape(-1, 1)])


In [ ]:
# Cross Validation
from sklearn.model_selection import StratifiedKFold

folds=range(1,11)
SVM = svm.SVC(C=1.0, kernel='linear',class_weight='balanced', gamma='auto',decision_function_shape='ovo',probability=True)

X=per_users_high['title_final']
X_2=per_users_high['side']
y=per_users_high['gender']
skf = StratifiedKFold(n_splits=10)

f1_ge_high=[]
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    X_2_train, X_2_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #Encoding
    Encoder = LabelEncoder()

    #word vectorization
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(data["title_final"])
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(X_test)
    #encoding the side
    Train_X_1_pos = Encoder.fit_transform(X_2_train)
    Test_X_1_pos = Encoder.fit_transform(X_2_test)

    #stacking everything together
    Train_X_Tfidf = hstack([Train_X_Tfidf, Train_X_1_pos.reshape(-1, 1)])
    Test_X_Tfidf = hstack([Test_X_Tfidf, Test_X_1_pos.reshape(-1, 1)])


    #gender
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)

    
    clf = SVM.fit(Train_X_Tfidf, y_train)
    y_pred = clf.predict(Test_X_Tfidf)
    f1_ge_high.append(f1_score(y_test,y_pred, average="weighted"))
    

    
    
f1_re_high=[]    
y=per_users_high['religion']
skf = StratifiedKFold(n_splits=10)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    X_2_train, X_2_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #Encoding
    Encoder = LabelEncoder()

    #word vectorization
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(data["title_final"])
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(X_test)
    #encoding the side
    Train_X_1_pos = Encoder.fit_transform(X_2_train)
    Test_X_1_pos = Encoder.fit_transform(X_2_test)

    #stacking everything together
    Train_X_Tfidf = hstack([Train_X_Tfidf, Train_X_1_pos.reshape(-1, 1)])
    Test_X_Tfidf = hstack([Test_X_Tfidf, Test_X_1_pos.reshape(-1, 1)])

    #religion
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)

    
    clf = SVM.fit(Train_X_Tfidf, y_train)
    y_pred = clf.predict(Test_X_Tfidf)

    f1_re_high.append(f1_score(y_test,y_pred, average="weighted"))    

f1_et_high=[]    
y=per_users_high['ethnicity']
skf = StratifiedKFold(n_splits=10)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    X_2_train, X_2_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #Encoding
    Encoder = LabelEncoder()

    #word vectorization
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(data["title_final"])
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(X_test)
    #encoding the side
    Train_X_1_pos = Encoder.fit_transform(X_2_train)
    Test_X_1_pos = Encoder.fit_transform(X_2_test)
    #stacking everything together
    Train_X_Tfidf = hstack([Train_X_Tfidf, Train_X_1_pos.reshape(-1, 1)])
    Test_X_Tfidf = hstack([Test_X_Tfidf, Test_X_1_pos.reshape(-1, 1)])

    #ethnicity
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)

    
    clf = SVM.fit(Train_X_Tfidf, y_train)
    y_pred = clf.predict(Test_X_Tfidf)

    f1_et_high.append(f1_score(y_test,y_pred, average="weighted")) 
    
    
f1_id_high=[]    
y=per_users_high['ideology']
skf = StratifiedKFold(n_splits=10)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    X_2_train, X_2_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #Encoding
    Encoder = LabelEncoder()

    #word vectorization
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(data["title_final"])
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Test_X_Tfidf = Tfidf_vect.transform(X_test)
    #encoding the side
    Train_X_1_pos = Encoder.fit_transform(X_2_train)
    Test_X_1_pos = Encoder.fit_transform(X_2_test)
    #stacking everything together
    Train_X_Tfidf = hstack([Train_X_Tfidf, Train_X_1_pos.reshape(-1, 1)])
    Test_X_Tfidf = hstack([Test_X_Tfidf, Test_X_1_pos.reshape(-1, 1)])

    #ideology
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)

    
    clf = SVM.fit(Train_X_Tfidf, y_train)
    y_pred = clf.predict(Test_X_Tfidf)

    f1_id_high.append(f1_score(y_test,y_pred, average="weighted"))     

